# Day 02: Methods - Why They Exist and When to Use Them

## 🚨 THE PROBLEM I'M TRYING TO SOLVE

I'm building a simple banking system. Here's what I need:

1. **Multiple accounts** - Alice, Bob, Charlie all need their own accounts
2. **Each account has a balance** - but it CANNOT go negative (validation needed!)
3. **Every transaction must be logged** - for security/auditing
4. **Interest rate validation** - same rule applies to ALL accounts (0-5% only)
5. **Minimum balance rule** - ALL accounts must maintain at least $100

**The nightmare without methods:**
- I'd write validation code 50 times
- I'd copy-paste logging everywhere
- If I change a rule, I'd have to update 50 places
- One typo = bugs everywhere

**The solution: Methods!**
Put the logic ONCE in the class. Every account uses it. Change once, works everywhere.

---

## My Learning Path

I'll solve this step-by-step:
1. Start with basic instance methods (deposit, withdraw)
2. Add validation (protected methods)
3. Add logging (private methods) 
4. Add class-level rules (class variables)
5. Add utility functions (static methods)
6. Add class-level actions (class methods)

Let's build this thing!


## Step 1: The Simplest Possible Bank Account

Okay, let me start SUPER simple. Just an account with a balance. No validation yet, just the basics.

**What I'm learning here:** This is an instance method - `deposit()` belongs to EACH account. When Alice deposits, it only affects Alice's account.


In [2]:
# Step 1: Basic instance method
# self = the specific account object (alice_account, bob_account, etc.)
# When I call alice_account.deposit(100), self IS alice_account

class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner  # Each account has its OWN owner
        self.balance = balance  # Each account has its OWN balance
    
    # This is an INSTANCE METHOD - it operates on ONE specific account
    # Notice: first parameter is ALWAYS 'self' (the account itself)
    def deposit(self, amount):
        self.balance += amount  # self.balance = THIS account's balance
        return self.balance
    
    def get_balance(self):
        return self.balance
    
    # __str__ makes printing nice - without it, I'd see <__main__.BankAccount object at 0x...>
    def __str__(self):
        return f"Account({self.owner}: ${self.balance})"


In [3]:
# Testing Step 1: Each account is independent!
# This is the KEY insight: methods operate on ONE instance at a time

alice_account = BankAccount("Alice", 500)
bob_account = BankAccount("Bob", 1000)

print(f"Before: {alice_account}")
print(f"Before: {bob_account}")

# Alice deposits - only HER balance changes
alice_account.deposit(200)
print(f"\nAfter Alice deposits $200:")
print(f"Alice: {alice_account}")
print(f"Bob: {bob_account}")  # Bob's balance unchanged!

# See? Each account has its OWN balance. Methods work on 'self' (the specific account)

Before: Account(Alice: $500)
Before: Account(Bob: $1000)

After Alice deposits $200:
Alice: Account(Alice: $700)
Bob: Account(Bob: $1000)


## Step 2: Adding Validation (Protected Methods)

**The problem:** Right now I can deposit negative amounts! That's broken. I need validation.

**The solution:** Create a helper method for validation. But I don't want users calling it directly - it's an internal detail.

**Protected methods** (single underscore `_method_name`): 
- Convention: "Hey, this is internal, don't use it from outside"
- Python doesn't enforce it, but it's a signal to other developers (and future me!)

**Why this matters:** I can change HOW validation works without breaking code that uses `deposit()`.

In [4]:
# Step 2: Adding validation with protected methods
# The _ prefix is a CONVENTION (not enforced by Python) - it says "internal use only"

class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance
    
    def deposit(self, amount):
        # Use the protected method for validation
        if self._is_valid_amount(amount):
            self.balance += amount
            return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def withdraw(self, amount):
        # Reuse the same validation method - DRY principle!
        if self._is_valid_amount(amount):
            if self.balance >= amount:
                self.balance -= amount
                return self.balance
            else:
                print(f"❌ Insufficient funds. Balance: ${self.balance}, Requested: ${amount}")
                return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    # PROTECTED METHOD (single underscore)
    # This is an internal helper - I can call it from other methods in this class
    # But I'm signaling "don't use this from outside the class"
    def _is_valid_amount(self, amount):
        return amount > 0
    
    def get_balance(self):
        return self.balance
    
    def __str__(self):
        return f"Account({self.owner}: ${self.balance})"

In [5]:
# Testing Step 2: Validation works!
# Notice how I reuse _is_valid_amount() in both deposit() and withdraw()
# This is the DRY principle: Don't Repeat Yourself

account = BankAccount("Alice", 500)

print("Testing validation:")
account.deposit(-100)  # Should fail
account.deposit(200)   # Should work
print(f"Balance: ${account.get_balance()}")

print("\nTesting withdraw:")
account.withdraw(50)   # Should work
account.withdraw(1000) # Should fail (insufficient funds)
account.withdraw(-50)  # Should fail (invalid amount)
print(f"Final balance: ${account.get_balance()}")

# I can still call _is_valid_amount() from outside (Python doesn't prevent it)
# But the underscore is a signal: "Hey, this is internal, don't rely on it!"
print(f"\nProtected method (not recommended to call directly): {account._is_valid_amount(100)}")

Testing validation:
❌ Invalid amount: $-100. Must be positive.
Balance: $700

Testing withdraw:
❌ Insufficient funds. Balance: $650, Requested: $1000
❌ Invalid amount: $-50. Must be positive.
Final balance: $650

Protected method (not recommended to call directly): True


## Step 3: Private Methods for Internal Details

**The problem:** I need to log every transaction, but logging is messy implementation detail. Users shouldn't care HOW I log.

**Private methods** (double underscore `__method_name`):
- Python DOES enforce this - it "mangles" the name to make it harder to access
- This is for things that are TRULY internal - like logging, formatting, etc.
- The name becomes `_ClassName__method_name` (name mangling)

**When to use:**
- `_single_underscore`: "Hey, this is internal, but you might need it" (protected)
- `__double_underscore`: "This is implementation detail, stay away!" (private)

In [6]:
# Step 3: Adding private methods for logging
# Notice: I'm updating the SAME class, building on what I have

class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance
    
    def deposit(self, amount):
        if self._is_valid_amount(amount):
            self.balance += amount
            self.__log_transaction(amount, "Deposit")  # Private method call
            return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def withdraw(self, amount):
        if self._is_valid_amount(amount):
            if self.balance >= amount:
                self.balance -= amount
                self.__log_transaction(amount, "Withdraw")  # Private method call
                return self.balance
            else:
                print(f"❌ Insufficient funds. Balance: ${self.balance}, Requested: ${amount}")
                return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    # PROTECTED: Internal helper, but might be useful
    def _is_valid_amount(self, amount):
        return amount > 0
    
    # PRIVATE: Implementation detail - Python name-mangles this!
    # The actual name becomes: _BankAccount__log_transaction
    # This makes it harder (but not impossible) to access from outside
    def __log_transaction(self, amount, operation):
        # In real code, this might write to a file, database, etc.
        # But users don't need to know HOW logging works
        print(f"📝 LOG: {operation} ${amount} | Account: {self.owner} | New Balance: ${self.balance}")
    
    def get_balance(self):
        return self.balance
    
    def __str__(self):
        return f"Account({self.owner}: ${self.balance})"


In [7]:
# Testing Step 3: Logging happens automatically!
# Notice how logging is "hidden" - users just call deposit/withdraw
# They don't need to know about __log_transaction

account = BankAccount("Alice", 500)
account.deposit(200)
account.withdraw(50)

# Try to access private method (it's name-mangled, so this won't work easily)
# account.__log_transaction(100, "Test")  # This would fail!
# But if I REALLY wanted to, I could do: account._BankAccount__log_transaction(100, "Test")
# But DON'T - that's breaking encapsulation!


📝 LOG: Deposit $200 | Account: Alice | New Balance: $700
📝 LOG: Withdraw $50 | Account: Alice | New Balance: $650


650

## Step 4: Class Variables (Shared Rules for ALL Accounts)

**The problem:** I need a minimum balance rule that applies to EVERY account. If I change it, it should change for ALL accounts.

**Class variables** (defined at class level, not in `__init__`):
- Shared by ALL instances of the class
- Change it once, affects everyone
- Perfect for rules, constants, configuration

**Why this matters:** Instead of hardcoding `100` everywhere, I define it ONCE. If the bank changes the rule, I update ONE line.


In [8]:
# Step 4: Adding class variable for minimum balance
# This is shared by ALL accounts - change it once, affects everyone!

class BankAccount:
    # CLASS VARIABLE - shared by ALL instances
    # Access it via: BankAccount.MIN_BALANCE or self.MIN_BALANCE
    MIN_BALANCE = 100  # This applies to EVERY account
    
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance
        # Check minimum balance when creating account
        if self.balance < BankAccount.MIN_BALANCE:
            print(f"⚠️ Warning: Balance ${self.balance} is below minimum ${BankAccount.MIN_BALANCE}")
    
    def deposit(self, amount):
        if self._is_valid_amount(amount):
            self.balance += amount
            self.__log_transaction(amount, "Deposit")
            return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def withdraw(self, amount):
        if self._is_valid_amount(amount):
            # Check minimum balance BEFORE withdrawing
            if self.balance - amount < BankAccount.MIN_BALANCE:
                print(f"❌ Cannot withdraw. Would go below minimum balance of ${BankAccount.MIN_BALANCE}")
                return self.balance
            elif self.balance >= amount:
                self.balance -= amount
                self.__log_transaction(amount, "Withdraw")
                return self.balance
            else:
                print(f"❌ Insufficient funds. Balance: ${self.balance}, Requested: ${amount}")
                return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def _is_valid_amount(self, amount):
        return amount > 0
    
    def __log_transaction(self, amount, operation):
        print(f"📝 LOG: {operation} ${amount} | Account: {self.owner} | New Balance: ${self.balance}")
    
    def get_balance(self):
        return self.balance
    
    def __str__(self):
        return f"Account({self.owner}: ${self.balance})"


In [9]:
# Testing Step 4: Class variable is shared!
# Notice: ALL accounts use the SAME MIN_BALANCE value

alice = BankAccount("Alice", 500)
bob = BankAccount("Bob", 50)  # Below minimum - warning!

print(f"\nAlice's balance: ${alice.balance}")
print(f"Bob's balance: ${bob.balance}")
print(f"Minimum balance (shared): ${BankAccount.MIN_BALANCE}")

# Try to withdraw too much - should fail due to minimum balance
print("\nAlice tries to withdraw $450 (would leave $50, below minimum):")
alice.withdraw(450)

# Change the class variable - affects ALL accounts!
print(f"\n⚠️ Bank changes policy! New minimum: $200")
BankAccount.MIN_BALANCE = 200
print(f"Now Alice can't withdraw as much:")
alice.withdraw(400)  # Would leave $100, but minimum is now $200!


⚠️ Warning: Balance $50 is below minimum $100

Alice's balance: $500
Bob's balance: $50
Minimum balance (shared): $100

Alice tries to withdraw $450 (would leave $50, below minimum):
❌ Cannot withdraw. Would go below minimum balance of $100

⚠️ Bank changes policy! New minimum: $200
Now Alice can't withdraw as much:
❌ Cannot withdraw. Would go below minimum balance of $200


500

## Step 5: Static Methods (Utility Functions)

**The problem:** I need to validate interest rates (0-5% only). This doesn't need account data - it's just a utility function.

**Static methods** (`@staticmethod`):
- Don't need `self` or `cls` - they're just functions that happen to live in the class
- Can be called on the class OR an instance
- Perfect for utility functions that are related to the class but don't need instance data

**Why this matters:** Keeps related functions organized. Instead of a random function floating around, it lives with the class it's related to.


In [10]:
# Step 5: Adding static method for interest rate validation
# Notice: NO 'self' parameter! It doesn't need account data.

class BankAccount:
    MIN_BALANCE = 100
    
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance
        if self.balance < BankAccount.MIN_BALANCE:
            print(f"⚠️ Warning: Balance ${self.balance} is below minimum ${BankAccount.MIN_BALANCE}")
    
    def deposit(self, amount):
        if self._is_valid_amount(amount):
            self.balance += amount
            self.__log_transaction(amount, "Deposit")
            return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def withdraw(self, amount):
        if self._is_valid_amount(amount):
            if self.balance - amount < BankAccount.MIN_BALANCE:
                print(f"❌ Cannot withdraw. Would go below minimum balance of ${BankAccount.MIN_BALANCE}")
                return self.balance
            elif self.balance >= amount:
                self.balance -= amount
                self.__log_transaction(amount, "Withdraw")
                return self.balance
            else:
                print(f"❌ Insufficient funds. Balance: ${self.balance}, Requested: ${amount}")
                return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def apply_interest(self, rate):
        # Use the static method to validate BEFORE applying
        if BankAccount.is_valid_interest_rate(rate):
            interest = self.balance * (rate / 100)
            self.balance += interest
            self.__log_transaction(interest, f"Interest ({rate}%)")
            return self.balance
        else:
            print(f"❌ Invalid interest rate: {rate}%. Must be between 0-5%.")
            return self.balance
    
    def _is_valid_amount(self, amount):
        return amount > 0
    
    def __log_transaction(self, amount, operation):
        print(f"📝 LOG: {operation} ${amount} | Account: {self.owner} | New Balance: ${self.balance}")
    
    # STATIC METHOD - no 'self', no 'cls', just a utility function
    # Can be called: BankAccount.is_valid_interest_rate(3) OR account.is_valid_interest_rate(3)
    @staticmethod
    def is_valid_interest_rate(rate):
        # This doesn't need ANY account data - it's just a validation rule
        return 0 <= rate <= 5
    
    def get_balance(self):
        return self.balance
    
    def __str__(self):
        return f"Account({self.owner}: ${self.balance})"


In [11]:
# Testing Step 5: Static methods work without an instance!
# I can call it on the CLASS (no account needed)

print("Testing static method:")
print(f"Is 3% valid? {BankAccount.is_valid_interest_rate(3)}")
print(f"Is 10% valid? {BankAccount.is_valid_interest_rate(10)}")
print(f"Is -1% valid? {BankAccount.is_valid_interest_rate(-1)}")

# I can ALSO call it on an instance (works the same)
account = BankAccount("Alice", 1000)
print(f"\nCalled on instance: {account.is_valid_interest_rate(4)}")

# Now test applying interest
print("\nApplying valid interest (3%):")
account.apply_interest(3)

print("\nTrying invalid interest (10%):")
account.apply_interest(10)  # Should fail


Testing static method:
Is 3% valid? True
Is 10% valid? False
Is -1% valid? False

Called on instance: True

Applying valid interest (3%):
📝 LOG: Interest (3%) $30.0 | Account: Alice | New Balance: $1030.0

Trying invalid interest (10%):
❌ Invalid interest rate: 10%. Must be between 0-5%.


1030.0

## Step 6: Class Methods (Class-Level Actions)

**The problem:** I want to change the minimum balance rule for ALL accounts. But I want to do it through a method (not directly accessing the variable).

**Class methods** (`@classmethod`):
- First parameter is `cls` (the class itself, not an instance)
- Can modify class variables
- Perfect for factory methods, configuration changes, class-level operations

**Why this matters:** Instead of `BankAccount.MIN_BALANCE = 200`, I can do `BankAccount.set_min_balance(200)`. More controlled, can add validation/logging.


In [12]:
# Step 6: Adding class method to change minimum balance
# Notice: first parameter is 'cls' (the class), not 'self' (an instance)

class BankAccount:
    MIN_BALANCE = 100
    
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance
        if self.balance < BankAccount.MIN_BALANCE:
            print(f"⚠️ Warning: Balance ${self.balance} is below minimum ${BankAccount.MIN_BALANCE}")
    
    def deposit(self, amount):
        if self._is_valid_amount(amount):
            self.balance += amount
            self.__log_transaction(amount, "Deposit")
            return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def withdraw(self, amount):
        if self._is_valid_amount(amount):
            if self.balance - amount < BankAccount.MIN_BALANCE:
                print(f"❌ Cannot withdraw. Would go below minimum balance of ${BankAccount.MIN_BALANCE}")
                return self.balance
            elif self.balance >= amount:
                self.balance -= amount
                self.__log_transaction(amount, "Withdraw")
                return self.balance
            else:
                print(f"❌ Insufficient funds. Balance: ${self.balance}, Requested: ${amount}")
                return self.balance
        else:
            print(f"❌ Invalid amount: ${amount}. Must be positive.")
            return self.balance
    
    def apply_interest(self, rate):
        if BankAccount.is_valid_interest_rate(rate):
            interest = self.balance * (rate / 100)
            self.balance += interest
            self.__log_transaction(interest, f"Interest ({rate}%)")
            return self.balance
        else:
            print(f"❌ Invalid interest rate: {rate}%. Must be between 0-5%.")
            return self.balance
    
    def _is_valid_amount(self, amount):
        return amount > 0
    
    def __log_transaction(self, amount, operation):
        print(f"📝 LOG: {operation} ${amount} | Account: {self.owner} | New Balance: ${self.balance}")
    
    @staticmethod
    def is_valid_interest_rate(rate):
        return 0 <= rate <= 5
    
    # CLASS METHOD - first parameter is 'cls' (the class itself)
    # Can modify class variables, create instances, etc.
    @classmethod
    def set_min_balance(cls, new_min):
        # Validate the new minimum
        if new_min < 0:
            print(f"❌ Invalid minimum balance: ${new_min}. Must be positive.")
            return
        old_min = cls.MIN_BALANCE
        cls.MIN_BALANCE = new_min
        print(f"✅ Minimum balance changed from ${old_min} to ${new_min} (affects ALL accounts)")
    
    # Another class method: factory method (creates accounts in a specific way)
    @classmethod
    def create_premium_account(cls, owner):
        # Premium accounts start with $1000 minimum
        return cls(owner, balance=1000)
    
    def get_balance(self):
        return self.balance
    
    def __str__(self):
        return f"Account({self.owner}: ${self.balance})"


In [13]:
# Testing Step 6: Class methods operate on the CLASS, not instances

# Create some accounts
alice = BankAccount("Alice", 500)
bob = BankAccount("Bob", 300)

print(f"Current minimum balance: ${BankAccount.MIN_BALANCE}")
print(f"Alice balance: ${alice.balance}")
print(f"Bob balance: ${bob.balance}")

# Use class method to change minimum balance (affects ALL accounts!)
print("\nUsing class method to change minimum balance:")
BankAccount.set_min_balance(250)

# Now both accounts use the new minimum
print(f"\nNew minimum: ${BankAccount.MIN_BALANCE}")
print("Alice tries to withdraw $300 (would leave $200, below new minimum):")
alice.withdraw(300)

# Test factory method (class method that creates instances)
print("\nCreating premium account using factory method:")
premium = BankAccount.create_premium_account("Charlie")
print(premium)


Current minimum balance: $100
Alice balance: $500
Bob balance: $300

Using class method to change minimum balance:
✅ Minimum balance changed from $100 to $250 (affects ALL accounts)

New minimum: $250
Alice tries to withdraw $300 (would leave $200, below new minimum):
❌ Cannot withdraw. Would go below minimum balance of $250

Creating premium account using factory method:
Account(Charlie: $1000)


## 🎯 THE WHOLE POINT OF METHODS - Why They Exist

Okay, I've built this whole banking system. Let me step back and understand WHY methods exist.

### The Problem Methods Solve

**Without methods**, I'd have to:
- Write validation code 50 times (once per account operation)
- Copy-paste logging everywhere
- Update 50 places if a rule changes
- Risk bugs from typos/inconsistencies

**With methods**, I:
- Write validation ONCE in `_is_valid_amount()`
- Write logging ONCE in `__log_transaction()`
- Change rules ONCE (class variables, class methods)
- All accounts automatically use the updated logic

### The Key Insight

**Methods = Reusable Behavior**

Instead of repeating code, I define behavior ONCE in the class. Every instance uses it. This is:
- **DRY** (Don't Repeat Yourself)
- **Maintainable** (change once, works everywhere)
- **Organized** (related code lives together)
- **Encapsulated** (hide messy details, expose clean API)

### Method Types - Quick Reference

1. **Instance methods** (`self`): Actions each account can do
   - `deposit()`, `withdraw()`, `get_balance()`
   - Operate on ONE specific account

2. **Protected methods** (`_method`): Internal helpers
   - `_is_valid_amount()`
   - Convention: "internal use, but accessible"

3. **Private methods** (`__method`): Implementation details
   - `__log_transaction()`
   - Name-mangled: truly internal

4. **Class variables**: Shared state for ALL instances
   - `MIN_BALANCE = 100`
   - Change once, affects everyone

5. **Static methods** (`@staticmethod`): Utility functions
   - `is_valid_interest_rate()`
   - No instance data needed, just related to the class

6. **Class methods** (`@classmethod`): Class-level actions
   - `set_min_balance()`, `create_premium_account()`
   - Operate on the class itself, not instances

### The Bottom Line

Methods let me organize code by **behavior**, not just data. They're the difference between:
- A pile of variables and functions (messy)
- A cohesive class with organized behavior (clean)

**That's it. That's why methods exist.**


## 💡 Quick Mental Model

**Think of methods like this:**

- **Instance method** = "This account can do X"
  - `alice.deposit(100)` → Alice's account deposits

- **Protected method** = "Internal helper, but you might need it"
  - `_is_valid_amount()` → Used internally, but accessible if needed

- **Private method** = "Implementation detail, stay away!"
  - `__log_transaction()` → How logging works is none of your business

- **Class variable** = "Rule that applies to EVERYONE"
  - `MIN_BALANCE = 100` → All accounts must follow this

- **Static method** = "Utility function related to this class"
  - `is_valid_interest_rate()` → Doesn't need account data, just validates

- **Class method** = "Action on the class itself"
  - `set_min_balance()` → Changes the rule for ALL accounts

**Remember:** Methods exist to avoid repetition and organize behavior. That's the whole point!
